In [2]:
import pandas as pd
import numpy as np
import re
import random
import sys

from keras import layers
from keras.models import Sequential
from keras import optimizers

Using TensorFlow backend.


# Preview

In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [4]:
import prepare as prep
prep.preprocess('json_songs.json','test')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


ChordTablesException: cannot access chord tables address for Chord with 0 pitches

In [5]:
with open('test','r') as f:
    training_data = f.read() #reminder that training_data is a huge ass string

In [13]:
print(training_data)

5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 5 9 0 - 7 11 2 - 0 4 7 - 4 7 10 0 - 5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 5 9 0 - 7 11 2 - 0 4 7 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 2 5 9 - 7 11 2 - 0 4 7 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 7 11 2 - 9 1 4 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 0 4 7 - 5 9 0 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 2 5 9 - 7 11 2 - 0 4 7 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 9 0 4 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 7 11 2 - 9 1 4 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 7 11 2 - 0 4 7 - 7 11 2 - 9 1 4 - 2 5 9 - 7 11 2 - 5 9 0 - 0 4 7 - 2 5 9 - 7 11 2 - 9 1 4 - 2 6 9 - 7 11 2 - 9 1 4 - 2 6 9 - 9 1 4 - 11 2 6 - 4 7 11 - 9 1 4 - 2 6 9 - 7 11 2 - 9 1 4 - 2 6 9 - 9 1 4 - 11 3 6 - 4 7 11 - 9 1 4 - 2 6 9 - 7 11 2 - 9 1 4 - 2 6 9 - 9 1 4 - 11 3 6 - 4 7 11 - 9 1 4 - 2 6 9 - 11 3 6 - 4 7 11 - 9 1 4 - 7 11 2 - 7 11 2 - 9 1 4 - 2 6 9 - 11 2 6 - 7 11 2 - 9 1 4 - 2 6 9 . 0 3 7 - 10 2 5 - 3 7 10 - 0 3 7 - 10 2 5 - 3 7 1

# Training

In [7]:
def vectorizing_seq (text, maxlen, step):    
    """
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param step: sample a new sequence every n steps
    :type  step: int
    :returns: (Numpy boolean array of shape 
                    (Number of sequences, maxlen, number of distinct character),
               Numpy boolean array of shape 
                    (Number of sequences, number of distinct character),
               dictionary mapping a character to its integer placeholder)
    :rtype:   (numpy.ndarray, 
               numpy.ndarray, 
               dict)     
    """
    
    sentences = [] # hold extracted sequences
    next_chars = [] # hold next characters for each corresponding sentence

    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])

    print('Number of sequences:', len(sentences))

    chars = sorted(list(set(text)))
    print('Unique characters:', len(chars))
    char_indices = dict((char, chars.index(char)) for char in chars)
    print('Vectorization...')

    # one hot encoding the characters into binary arrays
    x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
        
    return x, y, char_indices

In [151]:
def create_model(x, y, maxlen, epochs, chars):
    """
    Creates and trains a model.
    :param x: Numpy boolean array of shape 
                    (Number of sequences, maxlen, number of distinct character)
    :type  x: numpy.ndarray
    :param y: Numpy boolean array of shape 
                    (Number of sequences, number of distinct character)
    :type  y: numpy.ndarray
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param epochs: number of training iterations
    :type  epochs: int
    :param chars: list of unique characters
    :type  chars: list
    :returns: trained keras model
    :rtype:   keras.engine.sequential.Sequential
    """

#     model = Sequential()
#     model.add(layers.GRU(
#         32,
#         return_sequences=True,
#         input_shape=(maxlen, len(chars)))
#     )
#     model.add(layers.GRU(
#         64,
#         input_shape=(maxlen, len(chars)))
#     )
#     model.add(layers.Dense(
#         len(chars), 
#         activation='softmax')
#     )

    # start of my model attempt, it works decently well
    # - try commenting it out and using the previous one
    # - also try removing the Dropout layers
    # --------------------------------------------
    model = Sequential()
    model.add(layers.GRU(
        128,
        return_sequences=True,
        input_shape=(maxlen, len(chars))
    ))
    model.add(layers.Dropout(0.5))
    model.add(layers.GRU(64))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(
        len(chars),
        activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    # --------------------------------------------

    print(model.summary())

    # default model old params
    # --------------------------------
    #optimizer = optimizers.RMSprop(lr=0.01)
    #model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    # --------------------------------
    model.fit(x, y, batch_size=128, epochs=epochs)

    return (model)

In [152]:
def train_model_from_text(text, maxlen=6, step=12, epochs=10):
    """
    Given text, train the model.
    
    :param text: A string with all the text together.
    :type  text: str
    :param maxlen: the length of a sequence to extract as train
    :type  maxlen: int
    :param step: sample a new sequence every n steps
    :type  step: int
    :param epochs: number of training iterations
    :type  epochs: int
    :returns: (trained keras model,
               dictionary mapping characters to digit representations)
    :rtype:   (keras.engine.sequential.Sequential,
               dict)
    """
    
    x, y, char_indices = vectorizing_seq(text, maxlen, step)
    chars = list (char_indices.keys())
    model = create_model(x, y, maxlen, epochs, chars)
    
    return model, char_indices

In [153]:
def sample(preds, temperature=1.0):
    """
    Compute new probability distribution based on the temperature
    Higher temperature creates more randomness.
    
    :param preds: numpy array of shape (unique chars,), and elements sum to 1
    :type  preds: numpy.ndarray
    :param temperature: characterizes the entropy of probability distribution
    :type  temperature: float
    :returns: a number 0 to the length of preds - 1
    :rtype:   int
    """
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [154]:
def text_generate(model, text, char_indices, maxlen=4, temperature=1.0, textlen=10):
    """
    Generate text based on a model.
    
    :param model: trained keras model
    :type  model: keras.engine.sequential.Sequential
    :param text: lyrics
    :type  text: str
    :param char_indices: dictionary mapping a character to its integer placeholder
    :type  char_indices: dict
    :param maxlen: maximum length of the sequences
    :type  maxlen: int
    :param textlen: Number of characters of generated sequence
    :type  textlen: int
    """

    start_index = random.randint(0, len(text) - maxlen - 1) 
    generated_text = text[start_index: start_index + maxlen] 
    print('--- Generating with seed: "' + generated_text + '"')
    
    chars = list (char_indices.keys())
    
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(textlen):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1
        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]
        sys.stdout.write(next_char)

# def text_generate_from(model, start, maxlen=4, temperature=0.8, textlen=10):
#     print('---- Generating with temperature {}...'.format(temperature))
#     sys.stdout.write(start)
    
#     chars = list(char_indices.keys())
#     gen_text = start
    
#     for i in range(textlen):
#         sampled = np.zeros((1, maxlen, len(chars)))
#         for t, char in enumerate(gen_text):
#             sampled[0, t, char_indices[char]] = 1

#         preds = model.predict(sampled, verbose=0)[0]
#         next_idx = sample(preds, temperature)
#         next_char = chars[next_idx]
#         gen_text += next_char
#         gen_text = gen_text[1:]
#         sys.stdout.write(next_char)

## Sample Training and Text Generation

In [ ]:
# small amount for now so I can test if the code works
# train on entire dataset
# sample_text = all_text[:5000]

training_data

In [158]:
#maxlen = 4
maxlen = 60 # train on sequences of 30 characters (about 3 chords)

model, char_indices = train_model_from_text(
    training_data,
    maxlen=maxlen,
    #step=6,
    step=10, # jump over 10 characters (~ one chord) 
    #epochs=15
    epochs=2
)

Number of sequences: 55622
Unique characters: 13
Vectorization...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_39 (GRU)                 (None, 60, 128)           54528     
_________________________________________________________________
dropout_13 (Dropout)         (None, 60, 128)           0         
_________________________________________________________________
gru_40 (GRU)                 (None, 64)                37056     
_________________________________________________________________
dropout_14 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_25 (Dense)             (None, 13)                845       
Total params: 92,429
Trainable params: 92,429
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
55622/55622 [==============================] - 69s 1ms/st

In [160]:
text_generate(
    model, 
    training_data, 
    char_indices, 
    maxlen=maxlen,
    temperature=.5,
    textlen=200
)

--- Generating with seed: "- 10 2 5 - 0 4 7 - 9 0 4 - 2 6 9 - 5 8 0 - 0 4 7 - 0 4 7 - 7"
------ temperature: 0.5
- 10 2 5 - 0 4 7 - 9 0 4 - 2 6 9 - 5 8 0 - 0 4 7 - 0 4 7 - 7 11 2 - 0 4 7 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 0 4 7 - 5 9 0 - 7 11 2 - 10 2 5 - 7 11 2 - 2 6 9 - 5 9 0 - 0 4 7 - 5 9 0 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 5 

In [140]:
import time
model.save('run_{}.h5'.format(time.strftime("%Y%m%d-%H%M%S")))

In [149]:
# text_generate_from(
#     model,
#     '0 3 7',
#     maxlen=maxlen,
#     temperature=0.8,
#     textlen=100
# )

---- Generating with temperature 0.8...
0 3 7.1  61  02    32111   11141    321 1    184141     8  4  1  1 -4   1111  199 88-436 1 1481   3 -6 13